In [1]:
# Clean Slate for clean thoughts :)
# Stuff
import numpy as np
import cvxpy as cvx
from scipy import sparse
from imTools import *
from Taylor08 import *
from convexHullPlot import *

In [2]:
base = readImage('images/BrainProtonDensitySlice.png', (50,50))
target = readImage('images/DeformedBrainPD.png', (50,50))

(m, n, _) = base.shape

In [3]:
p = Taylor08(base, target)

Getting Coefficients...
Done. Time elapsed: 11.918673038482666  


Formatting Matrices...
Done. Total Time Elapsed:  12.230146884918213 




/usr/local/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/local/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:203: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


KeyboardInterrupt: 

In [ ]:
p

In [ ]:
# Let's do an example with a smaller tranformation space to start:
# To get lower convex hull coefficients just compare target to base:
window = 10
k = 4
sigma = 10
(Ax, Ay, Iz, b, C) = getConstraintCoeffs(target, base, window, k, sigma)

In [ ]:
p = np.zeros(38)
z = np.ones(m*n)*130
max(Ax @ C @ p[:19] + Ay @ C @ p[19:] -Iz @ z - b)
# (p1,z1) = TaylorNewtonStep((im, Ax, Ay, Iz, b, Dx, Dy, C), p, z, 1)

In [ ]:
# Convex Lower Hull Plot Example
window = 8
plt.imshow(target)
(x,y) = (0,0)
errorSurface = []
for i in range(-window, window):
    for j in range(-window, window):

        if ((x + i) < m and (x + i) >= 0 and (y+j) < n and (y+j) >=0):
            error = np.linalg.norm(target[x,y] - base[x + i, y + j], 3) ** 2
            print(error)
            errorSurface.append([x + i, y + j, error])

errorSurface = np.array(errorSurface)
# hull = ConvexHull(points = errorSurface, qhull_options='QJ')
# hull = ConvexHull(points = errorSurface)

plotLowerHull(errorSurface, 1)

In [ ]:
def TaylorNewtonStep(facetCoeffs, p, z, t):
    # minimizes t*z - sum(log(ax_i ci.T px + ay_i ci.T py - Izi.T z - bi)) wrt z, px, py
    (im, Ax, Ay, Iz, b, C) = facetCoeffs
    (m, n, _) = im.shape
    c = t*np.ones(m*n)
    
    L = int(len(p) / 2)
    
    # line search params:
    alpha = 0.3
    beta = 0.8
    eps =  1/10**6 # Not sure what to make this
    maxiters = 100
    count = 0
    
    # Perform Line Search:
    for iter in range(maxiters):
        
        count += 1
        A = (Ax @ C @ p[:3]) + (Ay @ C @ p[3:]) - Iz @ z
        s = b - A
        d = sparse.diags(s**(-1))
        d2 = d**2

        (Hp, Hz, D6) = getHessians(Ax, Ay, Iz, C, d2)
        (gp, gz) = getGradients(Ax, Ay, Iz, C, s, t)
        
        # Solve for Newton Step:
        g = np.concatenate((gp,gz), axis = 0)
        
        Dinv = sparse.linalg.inv(D6)
        Hprime = Hp - Hz.T @ Dinv @ Hz
        gpprime = (-1 * gp) - Hz.T @ Dinv @ (-1 * gz)
        
        # Use Block Diagonals and Schur Complement to solve the system:
        # dp = np.linalg.solve(Hp - Hz.T @ Dinv @ Hz, (-1*gp) - Hz.T @ Dinv @ (-1*gz)) 
        dp = np.linalg.solve(Hprime, gpprime)
        dz = sparse.linalg.inv(D6)@((-1*gz) - Hz @ dp)
        delta = np.concatenate((dp, dz), axis = 0)
#         gprime = gp - Hz.T @ Dinv @ gz
        
        g2 = np.concatenate((gpprime, gz), axis = 0)
                
        # Check Optimality Gap:
#         lambdasqr = -1 * dp.T @ Hprime @ dp
        lambdasqr = -1 * g.T @ delta
        print('lambdasqr/2 = ', lambdasqr/2, '\n\n')
        if lambdasqr / 2 < eps:
            break # if already eps-suboptimal
       
        # else: 
        tau = 1        
        
        # Check if z + tau * dz is feasible
        while max(Ax @ C @ (p + tau * dp)[:L] + \
                  Ay @ C @ (p + tau * dp)[L:] - \
                  Iz @ (z + tau * dz) - b) >= 0.0:

            # Update tau
            tau = beta * tau
        print('Feasible step\n\n')      
        
        # Want f(x + t*x_nt) < f(x) + t*alpha*g.T @ x_nt
        # Ensure step is 
#         while c.T @ (z + tau * dz) \
#             - sum(np.log(-1 * (Ax @ C @ (p + tau * dp)[:L] + \
#                                 Ay @ C @ (p + tau * dp)[L:] - \
#                                 Iz @ (z + tau * dz) - b))) \
#                                 > c.T @ z - sum(np.log(-1 * (Ax @ C @ p[:L] + \
#                                   Ay @ C @ p[L:] - \
#                                   Iz @ z - b))) + \
#                                   alpha * t * g.T @ delta:
#             # Update tau
#             tau = beta * tau  
            
        while c.T @ (tau * dz) - sum(np.log(-1 * (Ax @ C @ (p + tau * dp)[:L] + \
                Ay @ C @ (p + tau * dp)[L:] - Iz @ (z + tau * dz) - b))) \
                + sum(np.log(-1 * (Ax @ C @ p[:L] + Ay @ C @ p[L:] - Iz @ z - b))) - \
                alpha * tau * g.T @ delta > 0:
            # Update tau
            tau = beta * tau
            
        print('alpha optimal step?\n\n')
        
        # prints p newton step
        print('dp = ', tau * dp, '\n\n')
        # Update px, py, z
        p += tau * dp
        z += tau * dz
        
        print('p = ', p, '\n\n')
        
        # Show current tranformation
        imCur = firstOrderDeformImage(base, p[:L], p[L:])
        plt.imshow(imCur)
        plt.show()
                
    if count == maxiters:
        print('ERROR: MAXITERS reached.\n')
        p = 0
        z = 0
    
    return (p, z)

In [ ]:
t = 1
mu = 20
eps = 1/10**3

M = len(b)
p = np.zeros(38)

while True:
    (pStar, zStar) = TaylorNewtonStep((Ax, Ay, Iz, b, C), p, z, t)
    p = pStar
    z = zStar
    gap = M/t
    if gap <= eps:
        break
    t = mu*t

return p

In [ ]:
base.shape

In [ ]:
Ax.shape[1]

In [ ]:
A = (Ax @ C @ p[:3]) + (Ay @ C @ p[3:]) - Iz @ z
s = b - A + 10**(-8)
d = sparse.diags(s**(-1))
d2 = d**2

(Hp, Hz, D6) = getHessians(Ax, Ay, Iz, C, d2)
(gp, gz) = getGradients(Ax, Ay, Iz, C, s, t)

# Solve for Newton Step:
g = np.concatenate((gp,gz), axis = 0)
Dinv = sparse.linalg.inv(D6)
dp = np.linalg.solve(Hp - Hz.T @ Dinv @ Hz, (-1*gp) - Hz.T @ Dinv @ (-1*gz))
dz = sparse.linalg.inv(D6)@((-1*gz) - Hz @ dp)
delta = np.concatenate((dp, dz), axis = 0)

In [4]:
z = cvx.Variable(m*n)
px = cvx.Variable(19)
py = cvx.Variable(19)
C2 = []
kernels = getKernels((m,n), 4)
for x in range(n):
    for y in range(m):
        C2.append(gaussianD((x,y), kernels, 10))
C2 = np.array(C2)
objective = cvx.Minimize(np.ones(m*n).T @ z)
constraints = [Ax @ C2 @ px + Ay @ C2 @ py - Iz @ z - b <=0, C2 @ px <= 40 * np.ones(m*n), C2 @ px >= -40 * np.ones(m*n), C2 @ py <= 40 * np.ones(m*n), C2 @ py >= -40 * np.ones(m*n)]
# constraints = [Ax @ C2 @ px + Ay @ C2 @ py - Iz @ z - b <=0]

prob = cvx.Problem(objective, constraints)

result = prob.solve(solver=cvx.ECOS)

im = gaussianDeformImage(base, 10, 4, px.value, py.value)
plt.imshow(im)

NameError: name 'Ax' is not defined

In [ ]:
py.value

In [ ]:
C2.T@px.value

In [ ]:
im = gaussianDeformImage(base, 10, 4, p[:19], p[19:])
plt.imshow(im)

In [ ]:
base.shape

In [ ]:
kernels = getKernels((m,n), 4)

In [ ]:
z = cvx.Variable(m*n)
px = cvx.Variable(3)
py = cvx.Variable(3)

objective = cvx.Minimize(np.ones(m*n).T @ z)
constraints = [Ax @ C @ px + Ay @ C @ py - Iz @ z - b <=0, C @ px <= 40 * np.ones(m*n), C @ px >= -40 * np.ones(m*n), C @ py <= 40 * np.ones(m*n), C @ py >= -40 * np.ones(m*n)]
# constraints = [Ax @ C2 @ px + Ay @ C2 @ py - Iz @ z - b <=0]

prob = cvx.Problem(objective, constraints)

result = prob.solve(solver=cvx.ECOS)

In [ ]:
px.value

In [ ]:
py.value

In [ ]:
p1

In [ ]:
p4